[![Open In Colab](./colab-badge.png)](https://colab.research.google.com/github/subhacom/moose-notebooks/blob/main/Network_of_LIF_neurons.ipynb) 

If you are using `colab`, in a fresh runtime you need to run pip to install pymoose there. On the other hand, if you are running jupyter locally and have pymoose installed in that environment, skip the line below.

In [ ]:
# Only for colab
# !pip install pymoose --quiet

# Network of LIF neurons
**Credit: The original implementation of this example was created by Aditya Gilra in 2014 for the CAMP summer school at NCBS, Bangalore** based on `Ostojic, S. (2014). Two types of asynchronous activity in networks of excitatory and inhibitory spiking neurons. Nat Neurosci 17, 594-600.`

*Look up `moose-examples` on `github` for his original object-oriented implementation. Here we shall implement this model in a simple procedural way.*

Key parameters to play with synaptic coupling `J`, external input rate `nu_ext`, strength of inhibitory input relative to excitatory `ginh`.


Rewritten by Subhasis Ray based on the article
```
N. Brunel, “Dynamics of Sparsely Connected Networks of Excitatory and Inhibitory Spiking Neurons,” J Comput Neurosci, vol. 8, no. 3, pp. 183–208, May 2000, doi: 10.1023/A:1008925309027.
```

In [ ]:
# Imports
import numpy as np
import matplotlib.pyplot as plt
import moose

In [ ]:
# Initialize numpy random number generator
rng = np.random.default_rng(1)   # keep a constant seed for replicability

N = 10
N_E = int(0.8 * N)   # number of excitatory neurons
N_I = N - N_E   # number of inhibitory neurons

theta = 20e-3   # spiking threshold for all neurons
Vr = 10e-3   # reset voltage 10 mV

## Model A
Brunel calls the case where inhibitory and excitatory neurons have identical properties *model A*. Then considers the case where the two differ, and calls it *model B*.


The model consists of $N_{E}$ excitatory neurons and $N_{I}$ inhibitory neurons.

Each neuron receives $C$ random connections from other neurons.

The number of connections from each type of neurons is proportional to the population size.

So, the number of excitatory connections is $C_{E} = \epsilon \times N_{E}$

and the number of inhibitory connections is $C_{I} = \epsilon \times N_{I}$.

In addition, each neurons receives $C_{ext}$ excitatory connections from outside the network. Each of these external synapses transmit events from independent Poisson processes with rate $\nu_{ext}$.


In [ ]:
# while J > 0 is PSP amplitude for all excitatory synapses,
# PSP amplitude for inhibitory synapses is -gJ. We name g `ginh`
J = 1e-3   # efficacy of excitatory synapse
ginh = 1.0   # scale factor for inhibitory synapses
# "External synapses are activated by independent Poisson processes with rate nu_ext"
nu_ext = 100.0
EPSILON = 1.0 # 0.1   # fraction of inputs from each type
C_E = int(EPSILON * N_E)      # number of excitatory connections per neuron
C_I = int(EPSILON * N_I)      # number of inhibitory connections per neuron

C_EXT = C_E    # number of external excitatory connections per neuron

DELAY = 1e-3    # synaptic delay
tau_E = 20e-3   # time constant of excitatory neurons
tau_rp = 2e-3   # refractory period

Rm = tau_E # 20e6       # membrane resistance
Cm = 1.0

In [ ]:
# containers
if moose.exists('/sim'):
    moose.delete('/sim')

sim = moose.Neutral('/sim')
model = moose.Neutral(f'{sim.path}/model')
data = moose.Neutral(f'{sim.path}/data')

Create an array of leaky-integrate fire neurons with `N` entries. All moose objects can be arrays if you specify the size as a second argument to the constructor.

This will make `netA` an array / vector of LIF elements. While for single elements you access object attributes using dot notation like `{Object}.{field}`, to access the fields of a vector simultaneously, you have to do this via the `vec` attribute `{Object}.vec.{field}`.

So, while for a single `LIF` object you would set the threshold as `LIF.thresh = value`, for the vector object it should be  `LIF.vec.thresh = value`.

In [ ]:
netA = moose.LIF(f'{model.path}/networkA', N)
netA.vec.thresh = theta

# Time constant of a parallel RC circuit is tau = Rm * Cm
netA.vec.Cm = Cm
netA.vec.Rm = Rm

netA.vec.refractoryPeriod = tau_rp

# set the voltage to reset potential when the similation is initialized with `moose.reinit()`.
netA.vec.initVm = 0.0
netA.vec.Em = 0.0
netA.vec.vReset = Vr
netA.vec.thresh = theta

Now we need to set up external inputs to the neurons. Each neuron receives $C_{ext} = C_{E}$ input connections from independent Poisson processes, each with $\nu_{ext}$ rate.

In [ ]:
ext_input = moose.RandSpike(f'{model.path}/ext_input', C_EXT * N)

# The RandSpike object can generate random spikes with a given rate
# approximating a Poisson process
ext_input.vec.rate = nu_ext

Finally, we shall keep track of the spikes in the network using some a vector of `Table` objects.

In [ ]:
ext_input_tab = moose.Table(f'{data.path}/ext_input_tab', ext_input.numData)
moose.connect(ext_input, 'spikeOut', ext_input_tab, 'input', 'OneToOne')

Let us break down the above. First, we are creating a `Table` array with the same number of elements as the `RandSpike` object `ext_input`. The `numData` field stores the number of elements in a MOOSE array object. A single object just has `numData = 1`.

The `pymoose` interface also exposes the underlying vector as the attribute `vec` which is like a `sequence` (string, list, tuple, etc. are of the abstract data type sequence in Python, they all share some common properties). So, apart from `numData` attribute, you can also find the number of elements with `len(ext_input.vec)`.

Then we are connecting `spikeOut` field of `ext_input` to the table's `input` field with a `OneToOne` message. `spikeOut` is a source field of `RandSpike` and  `input` is a destination field of `Table`. A `OneToOne` message connects these fields of corresponding elements in two vector objects, i.e., the `spikeOut` output value from the first element in `ext_input` will go into the `input` field of the first element of `ext_input_tab`, the output from the second element of `ext_input` will go into the `input` of the second element of `ext_input_tab`, and so on.


Next, we collect the spikes from all the elements in `netA` into another `Table` array.

In [ ]:
spike_tab = moose.Table(f'{data.path}/spike_tab', netA.numData)
moose.connect(netA, 'spikeOut', spike_tab, 'input', 'OneToOne')

Also, we want to monitor the state variable `Vm` of a few neurons. While spikes are discrete events in time, `Vm` is a continuous variable and is updated at every timestep of the simulation. However, the changes are so small from timestep to timestep, that we can sample its value at a much slower rate than it is computed. So instead of pushing the value at every timestep of `Vm` computation, we can get the `Table` to pull the value of `Vm` from the `LIF` object at a much slower rate. This will reduce both computer time and the amount of redundant data storage.

In [ ]:
vm_tab = moose.Table(f'{data.path}/vm_tab', 10)   # record from just 10 random LIF elements

# Select random neurons from netA for recording Vm
vm_idx = rng.choice(netA.numData, replace=False, size=vm_tab.numData)

for ii, jj in enumerate(vm_idx):
  moose.connect(vm_tab.vec[ii], 'requestOut', netA.vec[jj], 'getVm')

Now we shall setup the connections in the network itself. First, we must create synapses on the neurons. We use a `SimpleSynHandler` which collects spike events arriving from all the presynaptic neurons in each timestep and sends out the combined value through `activationOut` field. This is passed on to the postsynaptic `LIF` neuron. Each event increments the `Vm` of the postsynaptic neuron by the `weight` value of the synapse.

In [ ]:
synh = moose.SimpleSynHandler(f'{model.path}/synh', netA.numData)
moose.connect(synh, 'activationOut', netA, 'activation', 'OneToOne')
# Each neuron gets
# C_E synaptic inputs from excitatory neurons
# C_I synaptic inputs from inhibitory neurons
# and C_EXT = C_E sexternal excitatory synaptic inputs
synh.vec.numSynapses = C_E + C_I + C_EXT


Out of the `N` elements in the `LIF` vector, we shall designate the first `N_E` as excitatory, and the rest `N_I` as inhibitory neurons. Whether a neuron is excitatory or inhibitory depends only on the synapse.

- The first `C_E` connections to each synapse will be from randomly selected neurons out of the first `N_E`.
- The next `C_I` connections will be from the next `N_I` inhibitory neurons.
- Finally, the last `C_EXT` connections will come from the corresponding external Poisson spike generators in `ext_input`.


In [ ]:
for post_idx in range(N):
  # Elements 0 to N_E of netA are the neurons we designated excitatory
  # The first C_E synapses are excitatory, the rest C_I are inhibitory
  syn_idx = 0

  print(f'Connecting {syn_idx} to {syn_idx + C_E} excitatory inputs')
  pre_indices = set(range(N_E))
  # Avoid self connection.
  # Unlike set.remove(), set.discard() does not raise an error if the element
  # is not in the set.
  pre_indices.discard(post_idx)
  for pre_idx in rng.choice(list(pre_indices), size=C_E):
    synapse = synh.vec[post_idx].synapse[syn_idx]
    synapse.weight = J
    synapse.delay = DELAY
    moose.connect(netA.vec[pre_idx], 'spikeOut', synapse, 'addSpike')
    syn_idx += 1
  print(f'Connecting {syn_idx} to {syn_idx + C_I} inhibitory inputs')
  # Elements N_E to N of netA are the N_I neurons we designated inhibitory
  pre_indices = set(range(N_E, N))
  pre_indices.discard(post_idx)  # avoid self connection
  for pre_idx in rng.choice(list(pre_indices), size=C_I):
    # Synapses C_E till the last are the C_I inhibitory synapses
    synapse = synh.vec[post_idx].synapse[syn_idx]
    synapse.weight = - ginh * J
    synapse.delay = DELAY
    moose.connect(netA.vec[pre_idx], 'spikeOut', synapse, 'addSpike')
    syn_idx += 1

  # Connect external inputs
  print(f'Connecting {syn_idx} to {syn_idx + C_EXT} external inputs'
  f' from {post_idx * C_EXT} to {(post_idx + 1) * C_EXT}')
  for pre_idx in range(post_idx * C_EXT, (post_idx + 1) * C_EXT):
    synapse = synh.vec[post_idx].synapse[syn_idx]
    synapse.weight = J
    moose.connect(ext_input.vec[pre_idx], 'spikeOut', synapse, 'addSpike')
    syn_idx += 1


To check that model is correctly wired up, we can look at the connections using `moose.showmsg()` function.
Comment these out when you have large number of neurons because it may stall the system.

In [ ]:
moose.showmsg(netA.vec[0])

In [ ]:
moose.showmsg(synh.vec[0].synapse[0])

Now we can simulate the model after initializing it.

In [ ]:
#dt = 1e-5
runtime = 1.0e-1
# for ii in range(10):
#   moose.setClock( ii, dt )
moose.reinit()
moose.start(runtime)


First, for sanity check, we should see if the external spikes are being generated. For this we create a raster plot of the spike times of 1000 randomly selected external input source. Each row in the raster plot represents one external spike train, and the horizontal axis represents time. For each spike event in a spike train, we put a mark in its row at the time of the event.

In [ ]:
fig, ax = plt.subplots()
for ii, ext_idx in enumerate(rng.choice(ext_input.numData,
                                        size=min(1000, ext_input.numData),
                                        replace=False)):
  spike_times = ext_input_tab.vec[ext_idx].vector
  ax.plot(spike_times, [ii] * len(spike_times), 'k|')
ax.set_xlabel('Time (s)')

We can plot the `Vm` serieses we recorded.

In [ ]:
vm_tab.vec[0].vector

In [ ]:
t = np.arange(0, runtime + vm_tab.dt/2, vm_tab.dt)
fig, ax = plt.subplots()
for tab in vm_tab.vec:
  ax.plot(t, tab.vector, label=tab.name)
  # break
ax.set_xlabel('Time (s)')
ax.set_ylabel('Vm (mV)')
ax.legend()

In [ ]:
fig, ax = plt.subplots()
for ii, idx in enumerate(rng.choice(netA.numData,
                                        size=min(100, netA.numData),
                                        replace=False)):
  spike_times = spike_tab.vec[idx].vector
  ax.plot(spike_times, [ii] * len(spike_times), 'k|')
ax.set_xlabel('Time (s)')